In [2]:
import pandas as pd
import numpy as np
import torch
import os
import json
from transformers import BartTokenizer, BartModel, BartForConditionalGeneration
import math

/private/b/bergler/h_gurush/tmp/jupyter-envs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)#CAUTION: RUN THIS C

In [4]:
device

device(type='cuda')

In [5]:
from tqdm import tqdm

In [ ]:
# Get Dataset

In [60]:
import json
def get_data_from_arXiv_pubMed():
    content={}
    articles={}
    
    articles_docs_abstract = {}
    with open("Dataset/arxiv-dataset/train.txt","r") as file:
        for line in file:
            try:
                article = json.loads(line.strip())
                articles_docs_abstract[" ".join(article['article_text'])] = " ".join(article.get('abstract_text'))
               
            except json.JSONDecodeError as e:
                print(f"Skipping malformed line: {e}")
    pubmedArticles_docs_abs={}
    with open("Dataset/pubmed-dataset/train.txt","r") as w:
        for index,line in enumerate(w.readlines()):
            
               
            content=json.loads(line)
            pubmedArticles_docs_abs[" ".join(content['article_text'])] =" ".join(list(content.get('abstract_text')))
          
                   
                
    return pubmedArticles_docs_abs,articles_docs_abstract

In [61]:
pubMed,articles_dict=get_data_from_arXiv_pubMed()

In [6]:
import pickle

In [ ]:
# articles_dict=pickle.load(open("article_dict.pickle","rb"))
#pubMed=pickle.load(open("pubmed.pickle","rb"))

In [9]:
# pickle.dump(articles_dict,open("article_dict.pickle","wb"))
# pickle.dump(pubMed,open("pubmed.pickle","wb"))

In [10]:
from transformers import BartTokenizer, BartModel, BartForConditionalGeneration


In [10]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

In [ ]:
#use config to see the details

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

In [19]:
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased").to

# Tokenize and preprocess the data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=5000)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [18]:
type(encoded_dataset["train"])

datasets.arrow_dataset.Dataset

In [ ]:
import re
def cleaned_sent(doc):
    for index in range(0,len(doc)):
        sent=re.sub(r"@xcite[0-9]+"," ",doc[index]) #remove citation placeholder
        sent=re.sub(r"@xmath([0-9]+)",r"formula\1",sent) #replace formula placholder
        doc[index]=sent
    return doc

In [13]:
tokens=None
import pickle
def preprocess_documents_with_striding(documents,tokenizer,model,striding_rate=0.2,max_tokens=1020):
    input_ids_list =[]
    striding_count=math.floor(max_tokens-(max_tokens*striding_rate))
    reduced_document=[]
    attention_mask=[]
    cleaned_docs=cleaned_sent(documents)
    tokens=[]
   
    with torch.no_grad():
        for doc in tqdm(cleaned_docs):
           
            # remove citations and replace equation placeholder with better placeholders
            try:
                
                tokens=tokenizer(doc,truncation=False,padding=True,return_tensors="pt")["input_ids"].to(device)
                
                loop_count=math.ceil(tokens.shape[1]/striding_count)
                if loop_count==0:
                    continue
                target_length_max=math.ceil(max_tokens/loop_count)
                #80%max is min
                
                target_length_min=math.ceil(target_length_max-(target_length_max*0.2))
                doc_summary=[]
                #tokens=tokenizer(" ".join(cleaned_doc),max_length=math.ceil(loop_count*max_tokens),padding="max_length",return_tensors="pt").to(device)
                #print(tokens["input_ids"].shape,loop_count)
                for i in range(0,loop_count):
                    
                    st_index=i*striding_count
                    end_index=st_index+max_tokens
                    #sequence=" ".join(cleaned_doc[st_index:end_index])
                    
                    #print("sent",len(cleaned_doc[st_index:end_index]))
                    #tokens=tokenizer(sequence,max_length=max_tokens,padding="max_length",return_tensors="pt",truncation=True)
                    #print(tokens["input_ids"][0,st_index:end_index].unsqueeze(0).shape)
                    gen_tokens=model.generate(tokens[0,st_index:end_index].unsqueeze(0),num_beams=4, max_length=target_length_max,min_length=target_length_min, early_stopping=True)
                    #print(gen_tokens)
                    #print(gen_tokens.shape)
                    #decode=tokenizer.decode(gen_tokens.squeeze(0),skip_special_tokens=True)
                    doc_summary.append(gen_tokens.squeeze(0).cpu())
                docss=torch.cat(doc_summary)
                #print("shape of op",docss.shape)
                if docss.shape[0]>1024:
                    
                    docss=docss[0:1024]
                if docss.shape[0]<1024:
                    #print("no error here 0")
                    docss = torch.cat((docss,torch.tensor([tokenizer.pad_token_id or 0] * int(1024 - docss.shape[0]))))
                    #print("no error here")
                #print(docss.shape)
                output_attention_mask = (docss != tokenizer.pad_token_id).long()
                reduced_document.append(docss)
                attention_mask.append(output_attention_mask)
                torch.cuda.empty_cache()
            except Exception as e:
                print("error",e)
                continue
    return torch.stack(reduced_document).to(device),torch.stack(attention_mask).to(device)
        
            
        
        
        


In [14]:
import re
def clean_abstract(abstracts):
    cleaned_abstracts=[]
    for i in abstracts:
        text=re.sub(r"@xcite[0-9]+", "",i)
        text=re.sub(r"@xmath([0-9]+)",r"equation\1",text)
        cleaned_abstracts.append(text)
    return cleaned_abstracts

In [70]:
from transformers import BartTokenizer
from tqdm import tqdm
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
striding_model=BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
def preprocess_data(articles_dict,max_input_length=1024):
    """
    Tokenize the input (scientific document) and target (abstract).
    """
    input_ids=[]
    attention_mask=[]
    target_ids=[]
    count=0
    #for i in tqdm(articles_dict):
        # if count!=2:
        #     count+=1
        #     continue
   
    document,atn_mask=preprocess_documents_with_striding(list(articles_dict.keys())[0:10000],tokenizer,striding_model)
    abstract=clean_abstract(list(articles_dict.values())[0:10000])
    print("abstract",len(abstract))


    #documents=preprocess_documents_with_striding(documents,tokenizer,striding_model)
    #abstracts=clean_abstract(abstracts)
    
    # inputs = tokenizer.batch_decode(
    #     document, 
    #     max_length=max_input_length, 
    #     truncation=True, 
    #     padding="max_length", 
    #     return_tensors="pt"
    # )
    inputs={"input_ids":document,"attention_mask":atn_mask}
    
    targets = tokenizer(
        abstract, 
        max_length=max_input_length, 
        truncation=True, 
        padding="max_length", 
        return_tensors="pt"
    ).to(device)
    #targets=None
    # input_ids.append(inputs["input_ids"][0])
    # attention_mask.append(inputs["attention_mask"][0])
    # target_ids.append(targets["input_ids"][0])
    # inputs={"input_ids":torch.stack(input_ids,dim=0).to(device),"attention_mask":torch.stack(attention_mask,dim=0).to(device)}
    # targets={"input_ids":torch.stack(target_ids,dim=0).to(device)}
    return inputs, targets

In [16]:
from torch.utils.data import Dataset

class SummarizationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs["input_ids"][idx],
            'attention_mask': self.inputs["attention_mask"][idx],
            'labels': self.targets["input_ids"][idx]
        }

In [275]:
import multiprocessing
# inputs=[]
# targets=[]
# with multiprocessing.Pool(processes=2) as pool:
#     # Apply tqdm for tracking
#     for inp,target in tqdm(pool.imap(preprocess_data, articles_dict), total=len(articles_dict)):
#         inputs.extend(inp)
#         targets.extend(target)
torch.cuda.empty_cache()
inputs, targets = preprocess_data(articles_dict)
pickle.dump(inputs,open("articles_inputs.pickle","wb"))
pickle.dump(targets,open("articles_targets.pickle","wb"))
inputs1, targets1 = preprocess_data(pubMed)
pickle.dump(inputs1,open("pubMed_inputs.pickle","wb"))
pickle.dump(targets1,open("pubMed_targets.pickle","wb"))
dataset = SummarizationDataset(inputs, targets)

 79%|███████▉  | 7924/10000 [23:38:24<5:48:44, 10.08s/it] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 10000/10000 [29:57:02<00:00, 10.78s/it] 
